# Grupo 1 - Smog predicition
## Modelo AdaBoost

### Análisis y limpieza de datos

In [1]:
#Imports generales
import numpy as np
import pandas as pd
from pandas import Series
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

#Imports específicos
from sklearn import ensemble
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.metrics import classification_report, make_scorer, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import sem

#Visualización
import seaborn as sns
sns.set(color_codes=True)

%matplotlib inline

df = pd.read_csv('data/train.csv')

df['Gears'] = df['Transmission'].str.extract('(\d+)')
df['Gears'] = pd.to_numeric(df['Gears'], errors='coerce')
df['Transmission'] = df['Transmission'].str.extract('(\D+)')

In [2]:
#Fuel Type
df.loc[df["Fuel Type"] == "X", "Fuel Type"] = 0
df.loc[df["Fuel Type"] == "Z", "Fuel Type"] = 1
df.loc[df["Fuel Type"] == "D", "Fuel Type"] = 2
df.loc[df["Fuel Type"] == "E", "Fuel Type"] = 3
df.loc[df["Fuel Type"] == "N", "Fuel Type"] = 4

#Transmission
df.loc[df["Transmission"] == "A", "Transmission"] = 0
df.loc[df["Transmission"] == "AM", "Transmission"] = 1
df.loc[df["Transmission"] == "AS", "Transmission"] = 2
df.loc[df["Transmission"] == "AV", "Transmission"] = 3
df.loc[df["Transmission"] == "M", "Transmission"] = 4


#Vehicle Class
df.loc[df["Vehicle Class"] == "Compact", "Vehicle Class"] = 0
df.loc[df["Vehicle Class"] == "Full-size", "Vehicle Class"] = 1
df.loc[df["Vehicle Class"] == "Mid-size", "Vehicle Class"] = 2
df.loc[df["Vehicle Class"] == "Minicompact", "Vehicle Class"] = 3
df.loc[df["Vehicle Class"] == "Minivan", "Vehicle Class"] = 4
df.loc[df["Vehicle Class"] == "Minicompact", "Vehicle Class"] = 5
df.loc[df["Vehicle Class"] == "Pickup truck: Small", "Vehicle Class"] = 6
df.loc[df["Vehicle Class"] == "Pickup truck: Standard", "Vehicle Class"] = 7
df.loc[df["Vehicle Class"] == "SUV: Small", "Vehicle Class"] = 8
df.loc[df["Vehicle Class"] == "SUV: Standard", "Vehicle Class"] = 9
df.loc[df["Vehicle Class"] == "Special purpose vehicle", "Vehicle Class"] = 10
df.loc[df["Vehicle Class"] == "Station wagon: Mid-size", "Vehicle Class"] = 11
df.loc[df["Vehicle Class"] == "Station wagon: Small", "Vehicle Class"] = 12
df.loc[df["Vehicle Class"] == "Subcompact", "Vehicle Class"] = 13
df.loc[df["Vehicle Class"] == "Two-seater", "Vehicle Class"] = 14

#Gears
df['Gears'] = df['Gears'].fillna(df['Gears'].mean())

In [3]:
df.drop("Model Year", axis=1, inplace=True)
df.drop("Make", axis=1, inplace=True)
df.drop("Model", axis=1, inplace=True)
df.drop("Comb (mpg)", axis=1, inplace=True)
df.drop("Fuel Consumption City (L/100 km)", axis=1, inplace=True)
df.drop("Hwy (L/100 km)", axis=1, inplace=True)

df.head()

,id,Vehicle Class,Engine Size (L),Cylinders,Transmission,Fuel Type,Comb (L/100 km),CO2 Emissions (g/km),Smog,Gears
0,ab44e9bec15,12,2.0,4,1,1,8.7,202,2,7.0
1,45926762371,2,2.0,4,2,0,7.7,181,4,6.0
2,e9be56e153f,1,2.9,6,1,1,11.7,274,2,8.0
3,077092760df,0,2.0,4,2,0,8.1,189,1,6.0
4,c1c2579b795,3,5.2,12,0,1,13.8,324,1,8.0


Con esto, hemos terminado el análisis y la limpieza de los datos de ambos dataframes.

# Definión y entrenamiento del modelo

En este *notebook* realizaremos el entrenamiento de un modelo de AdaBoost, el cual entrenaremos empleando los campos 'Vehicle Class', 'Engine Size (L)', 'Cylinders', 'Transmission', 'Fuel Type', 'CO2 Emissions (g/km)' y 'Gears'.

En primer lugar separaremos el conjunto de datos en subconjuntos de entrenamiento y prueba para realizar una primera aproximación.

In [4]:
features = ['Vehicle Class', 'Engine Size (L)', 'Cylinders', 'Transmission', 'Fuel Type', 'Comb (L/100 km)', 'CO2 Emissions (g/km)', 'Gears']

x = df[features].values
y = df['Smog'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=33)

# Normalización
scaler = preprocessing.StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

Y una vez seleccionados los datos, entrenamos el modelo.

Utilizamos RandomForest como modelo interno de AdaBoost debido a que hemos probado varios modelos como KNN, SVM y DecisionTree entre otros pero RandomForest es el que mejor resultado nos ha dado con diferencia.

In [5]:
rfc = RandomForestClassifier(n_estimators=70, random_state=100)
 
model = AdaBoostClassifier(base_estimator=rfc, n_estimators=90, learning_rate=0.01, random_state=33)
 
model.fit(x_train, y_train)

AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=70,
                                                         random_state=100),
                   learning_rate=0.01, n_estimators=90, random_state=33)

Inicialmente se usan estos hiperparámetros para el entrenamiento. Posteriormente los ajustaremos para obtener una mayor eficiencia.

### Comprobación de resultados

Para la comprobación de resultados se van a calcular varias métricas, obteniendo la eficacia de nuestro modelo.

In [6]:
# Evaluar la Exactitud en el entrenamiento
predicted = model.predict(x_test)
expected = y_test

y_train_pred = model.predict(x_train)
print("Accuracy in training", metrics.accuracy_score(y_train, y_train_pred))

# También vamos a evaluar el error en las pruebas
y_test_pred = model.predict(x_test)
print("Accuracy in testing ", metrics.accuracy_score(y_test, y_test_pred))

Accuracy in training 0.9977220956719818
Accuracy in testing  0.7619047619047619


Obtenemos una exactitud de casi el 77%.

In [7]:
s_y_test = Series(y_test)
s_y_test.value_counts()

s_y_test.value_counts().head(1) / len(y_test)

2    0.408163
dtype: float64

Al cumplir la exactitud nula podemos deducir que el modelo no se encuentra desbalanceado en nuestro conjunto de datos.

Vamos con el F1-score:

In [8]:
print(classification_report(expected, predicted))

              precision    recall  f1-score   support

           0       1.00      0.86      0.92         7
           1       0.68      0.75      0.71        28
           2       0.76      0.75      0.76        60
           3       0.88      0.79      0.83        28
           4       0.69      0.75      0.72        24

    accuracy                           0.76       147
   macro avg       0.80      0.78      0.79       147
weighted avg       0.77      0.76      0.76       147



Ahora, vamos a probar a entrenarlo y evaluarlo con K-Fold para ver si puede mejorar los resultados obtenidos.

In [9]:
cv = KFold(n_splits=5, shuffle=True, random_state=33)

scores = cross_val_score(model, x, y, cv=cv)
print("Scores in every iteration", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores in every iteration [0.81355932 0.7008547  0.7008547  0.74358974 0.77777778]
Accuracy: 0.75 (+/- 0.09)


Utilizando KFold se obtiene una exactitud del 75%. 

### Ajuste del algoritmo

Se va a utilizar Grid Search para realizar una optimización de hiperparámetros.

In [10]:
# Conjunto de hiperparámetros a probar
tuned_hyperparameters = [{'n_estimators':  [10, 30, 50, 70, 90],
                          'learning_rate': [0.01, 0.1, 1.0],
                          'base_estimator': [rfc],
                          'random_state': [33]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Ajuste de hiperparámetros para %s" % score)
    print()

    if score == 'precision':
        scorer = make_scorer(precision_score, average='weighted', zero_division=0)
    elif score == 'recall':
        scorer = make_scorer(recall_score, average='weighted', zero_division=0)

    gs = GridSearchCV(AdaBoostClassifier(), tuned_hyperparameters, cv=10, scoring=scorer)
    gs.fit(x_train, y_train)

    print("Mejor conjunto de hiperparámetros encontrado en el conjunto de desarrollo:")
    print()
    print(gs.best_params_)
    print()
    print("Puntuaciones en la cuadrícula en el conjunto de desarrollo:")
    print()
    means = gs.cv_results_['mean_test_score']
    stds = gs.cv_results_['std_test_score']

    for mean_score, std_score, params in zip(means, stds, gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) para %r" % (mean_score, std_score * 2, params))
    print()

    print("Informe de clasificación detallado:")
    print()
    print("El modelo se entrena en el conjunto de desarrollo completo.")
    print("Las puntuaciones se calculan en el conjunto de evaluación completo.")
    print()
    y_true, y_pred = y_test, gs.predict(x_test)
    print(classification_report(y_true, y_pred))
    print()

# Ajuste de hiperparámetros para precision

Mejor conjunto de hiperparámetros encontrado en el conjunto de desarrollo:

{'base_estimator': RandomForestClassifier(n_estimators=70, random_state=100), 'learning_rate': 0.01, 'n_estimators': 10, 'random_state': 33}

Puntuaciones en la cuadrícula en el conjunto de desarrollo:

0.756 (+/-0.133) para {'base_estimator': RandomForestClassifier(n_estimators=70, random_state=100), 'learning_rate': 0.01, 'n_estimators': 10, 'random_state': 33}
0.750 (+/-0.113) para {'base_estimator': RandomForestClassifier(n_estimators=70, random_state=100), 'learning_rate': 0.01, 'n_estimators': 30, 'random_state': 33}
0.748 (+/-0.117) para {'base_estimator': RandomForestClassifier(n_estimators=70, random_state=100), 'learning_rate': 0.01, 'n_estimators': 50, 'random_state': 33}
0.743 (+/-0.127) para {'base_estimator': RandomForestClassifier(n_estimators=70, random_state=100), 'learning_rate': 0.01, 'n_estimators': 70, 'random_state': 33}
0.743 (+/-0.127) para {'b

### Comprobación con el algoritmo ajustado

A partir de los resultados anteriores, volvemos a entrenar el modelo mediante validación con K-Fold para comprobar la nueva media de puntuación.

In [11]:
model = Pipeline([
        ('scaler', StandardScaler()),
        ('ab', AdaBoostClassifier(**gs.best_params_))
])

model.fit(x_train, y_train) 

cv = KFold(10, shuffle=True, random_state=33)

scores = cross_val_score(model, x, y, cv=cv)
def mean_score(scores):
    return ("Puntuación media: {0:.3f} (+/- {1:.3f})").format(np.mean(scores), sem(scores))
print(mean_score(scores))


Puntuación media: 0.763 (+/- 0.023)


Tras el ajuste de hiperparámetros vemos que se obtiene una puntuación muy similar a la obtenida anteriormente.